<a href="https://colab.research.google.com/github/Parv17k/nyc_collision_data_analysis/blob/main/data_cleaning_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz


In [3]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [4]:
mv spark spark.tgz

mv: cannot stat 'spark': No such file or directory


In [5]:
ls spark-3.1.1-bin-hadoop2.7

ls: cannot access 'spark-3.1.1-bin-hadoop2.7': No such file or directory


In [6]:
!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [8]:
import findspark
findspark.init()

Exception: ignored

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

In [ ]:
mv rows.csv col.csv


In [ ]:
ls


In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [ ]:
spark =SparkSession(sc)

In [ ]:
col=spark.read.option("header",True).csv("./col.csv",inferSchema=True)
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True)

In [ ]:
col.count()-col.distinct().count()

In [ ]:
len(col.columns)


In [ ]:
col.printSchema()

In [ ]:
import seaborn as sns

In [ ]:
#dropping columns with collision id = null

In [ ]:
col.filter(col.COLLISION_ID.isNull()).drop()

In [ ]:
col.count()

In [ ]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [ ]:
cannotRetriveLocation =missingLocation.filter(col["LONGITUDE"].isNull())

In [ ]:
cannotRetriveLocation.count()

In [ ]:
# Infer the schema, and register the DataFrame as a table.

col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [ ]:
colNew= spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null ");


In [ ]:
colNew.count()

In [ ]:
col.count() - cannotRetriveLocation.count()

In [ ]:
colNew.filter(colNew["ZIP CODE"].isNull()).filter(colNew["BOROUGH"].isNotNull()).count()

In [ ]:
BOROUGH=list(map(lambda x : x["BOROUGH"],colNew.select("BOROUGH").distinct().collect()))

In [ ]:
BOROUGH

In [ ]:
longlat.printSchema()

In [ ]:

colNew.createOrReplaceTempView("colNew");
longlat.createOrReplaceTempView("longlat");

In [ ]:
missingZipCode=colNew.filter(colNew['ZIP CODE'].isNull());


In [ ]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [ ]:
for m in missingZipCode.limit(19).collect():
  zip=spark.sql("select zip,city FROM longlat where ROUND(longlat.latitude,2) ="+str(round(m.LATITUDE,2))+" and ROUND(longlat.longitude,2)= "+str(round(m.LONGITUDE,2))+" limit 1");
  zip=zip.select("zip","city").collect()
  if len(zip)>0:
    if zip[0].city.upper()=="NEW YORK":
      print(m["COLLISION_ID"],"MANHATTAN",zip[0].zip)
    else:
        if zip[0].city.upper() in BOROUGH:
          print(m["COLLISION_ID"],zip[0].city.upper(),zip[0].zip)